In [10]:
import pandas as pd
import numpy as np

test = pd.read_csv("/Users/jialing/Machine_Learning_Competitions/Predicting_Road_Accident_Risk/test (1).csv")
test.drop(columns=["id"], inplace=True)
test.head(10)

,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents
0,highway,2,0.34,45,night,clear,True,True,afternoon,True,True,1
1,urban,3,0.04,45,dim,foggy,True,False,afternoon,True,False,0
2,urban,2,0.59,35,dim,clear,True,False,afternoon,True,True,1
3,rural,4,0.95,35,daylight,rainy,False,False,afternoon,False,False,2
4,highway,2,0.86,35,daylight,clear,True,False,evening,False,True,3
5,highway,2,0.52,25,night,rainy,True,True,morning,False,True,1
6,rural,4,0.17,70,dim,clear,True,False,afternoon,True,True,0
7,urban,3,0.62,25,daylight,clear,False,True,afternoon,True,False,0
8,highway,3,0.93,45,dim,rainy,True,True,evening,True,False,2
9,urban,1,0.97,35,daylight,clear,True,True,evening,False,True,2


In [11]:
train = pd.read_csv("/Users/jialing/Machine_Learning_Competitions/Predicting_Road_Accident_Risk/train (1).csv")
train.drop(columns=["id"], inplace=True)
train.head(10)

,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56
5,highway,3,0.54,70,night,foggy,True,False,morning,False,True,0,0.61
6,highway,1,0.04,45,night,foggy,False,True,morning,True,False,2,0.20
7,highway,2,0.33,45,daylight,clear,False,True,evening,False,True,0,0.16
8,rural,3,0.63,25,dim,clear,True,False,evening,True,True,2,0.18
9,urban,4,0.38,45,dim,clear,True,True,afternoon,True,True,2,0.15


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517754 entries, 0 to 517753
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   road_type               517754 non-null  object 
 1   num_lanes               517754 non-null  int64  
 2   curvature               517754 non-null  float64
 3   speed_limit             517754 non-null  int64  
 4   lighting                517754 non-null  object 
 5   weather                 517754 non-null  object 
 6   road_signs_present      517754 non-null  bool   
 7   public_road             517754 non-null  bool   
 8   time_of_day             517754 non-null  object 
 9   holiday                 517754 non-null  bool   
 10  school_season           517754 non-null  bool   
 11  num_reported_accidents  517754 non-null  int64  
 12  accident_risk           517754 non-null  float64
dtypes: bool(4), float64(2), int64(3), object(4)
memory usage: 37.5+ MB


In [13]:
# Handling categorical features
cat_feat = ["road_type", "lighting", "weather", "time_of_day"]
bool_feat = ["road_signs_present", "public_road", "holiday", "school_season"]
num_feat = ['num_lanes', "curvature", "speed_limit", "num_reported_accidents"]

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer
from sklearn.model_selection import train_test_split

# Numeric bool_feat
for col in bool_feat:
    train[col] = train[col].apply(lambda x: 1 if x == True else 0)

for col in bool_feat:
    print(train[col].value_counts())

road_signs_present
0    259289
1    258465
Name: count, dtype: int64
public_road
1    260045
0    257709
Name: count, dtype: int64
holiday
1    260688
0    257066
Name: count, dtype: int64
school_season
0    260164
1    257590
Name: count, dtype: int64


In [17]:
# models for regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor 
from sklearn.svm import SVR

# Cat_feat using onehotencoder
train.dropna(inplace=True)

num_transformer = StandardScaler()
cat_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_feat), # we wont scale bools
        ('cat', cat_transformer, cat_feat)],
        remainder = 'passthrough')

X = train.drop(columns=["accident_risk"])
y = train["accident_risk"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training set shape:", X_train.shape, y_train.shape)
print(f"Val test shape:", X_val.shape, y_val.shape)

models = {'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
         'XGBRegressor': XGBRegressor(n_estimators=100, random_state=42),
         #'LGBMRegressor': LGBMRegressor(n_estimators=100, random_state=42),
         #'CatBoostRegressor': CatBoostRegressor(iterations=100, random_state=42, verbose=0),
         'GradientBoostingRegressor': GradientBoostingRegressor(n_estimators=100, random_state=42),
         'LinearRegression': LinearRegression(),
         'Ridge': Ridge(alpha=1.0),
         'Lasso': Lasso(alpha=0.1),
         }


for name, model in models.items():
    try:
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)])
        
        pipeline.fit(X_train, y_train)
        print(f"Model training complete for : {name}")
        
        from sklearn.model_selection import cross_val_score
        scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

        # 6. Convert negative MSE to positive, and calculate RMSE
        mse_scores = -scores
        rmse_scores = np.sqrt(mse_scores)

        print("\nCross-Validation RMSEs:", rmse_scores)
        print("Mean RMSE:", rmse_scores.mean())
        
        y_pred = pipeline.predict(X_val)
        mse = mean_squared_error(y_val, y_pred)
        mae = mean_absolute_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        
        print(f"Validation MSE: {mse:,.2f}")
        print(f"Validation MAE: {mae:,.2f}")
        print(f"Validation R2 Score: {r2:.2f}")
        print("--------------------------------------------------------")

    except Exception as e:
        print(f"An error occurred while training {name}: {e}")

Training set shape: (414203, 12) (414203,)
Val test shape: (103551, 12) (103551,)
Model training complete for : RandomForest

Cross-Validation RMSEs: [0.05918978 0.05947766 0.05926359 0.05909468 0.0589225 ]
Mean RMSE: 0.05918964330063348
Validation MSE: 0.00
Validation MAE: 0.05
Validation R2 Score: 0.87
--------------------------------------------------------
Model training complete for : XGBRegressor

Cross-Validation RMSEs: [0.05633119 0.05628807 0.05625263 0.05603279 0.05600349]
Mean RMSE: 0.05618163332162961
Validation MSE: 0.00
Validation MAE: 0.04
Validation R2 Score: 0.89
--------------------------------------------------------
Model training complete for : GradientBoostingRegressor

Cross-Validation RMSEs: [0.05688773 0.05705978 0.05683608 0.05678762 0.0566261 ]
Mean RMSE: 0.05683946112208097
Validation MSE: 0.00
Validation MAE: 0.04
Validation R2 Score: 0.88
--------------------------------------------------------
Model training complete for : LinearRegression

Cross-Validati

We will select XGBRegressor as our model for predicting road accident risk.

In [23]:
from sklearn.model_selection import GridSearchCV
best_model = XGBRegressor(n_estimators=100, random_state=42, learning_rate=0.1, max_depth=5)

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

param_dist = {
    'model__n_estimators': randint(100, 500),
    'model__max_depth': randint(3, 10),
    'model__learning_rate': uniform(0.01, 0.3),
    'model__subsample': uniform(0.6, 0.4),
    'model__colsample_bytree': uniform(0.6, 0.4)
}

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', best_model)
])
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=20,  # try 20 combinations
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

random_search.fit(X_train, y_train)
print("Best parameters:", random_search.best_params_)
print("Best RMSE:", np.sqrt(-random_search.best_score_))

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best parameters: {'model__colsample_bytree': np.float64(0.7797802696552814), 'model__learning_rate': np.float64(0.038623034947123394), 'model__max_depth': 9, 'model__n_estimators': 161, 'model__subsample': np.float64(0.7300733288106989)}
Best RMSE: 0.056068463723425915


In [ ]:
best_model = XGBRegressor(n_estimators=100, random_state=42, learning_rate=0.1, max_depth=5)

try:
    pipelinee = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', best_model)])
        
    pipeline.fit(X_train, y_train)
    print(f"Model training complete")
    y_pred_val = pipeline.predict(X_val)
    mse = mean_squared_error(y_val, y_pred_val)
    mae = mean_absolute_error(y_val, y_pred_val)
    r2 = r2_score(y_val, y_pred_val)
        
    print(f"Validation RMSE: {np.sqrt(mse):,.2f}")
    print(f"Validation MAE: {mae:,.2f}")
    print(f"Validation R2 Score: {r2:.2f}")
    print("--------------------------------------------------------")

        # Apply pipeline to test
    y_pred = pipeline.predict(test)
    print(y_pred)

except Exception as e:
    print(f"An error occurred while training {name}: {e}")



Model training complete
[0.2917683  0.12593062 0.18794118 ... 0.2506993  0.12877297 0.4857189 ]


In [ ]:
submission = pd.DataFrame({
    "id" : np.arange(517754, 517754+len(y_pred)),
    "accident_risk": y_pred
})
submission.to_csv("road_submission.csv", index=False)

submission
    

,id,accident_risk
0,517754,0.291768
1,517755,0.125931
2,517756,0.187941
3,517757,0.333810
4,517758,0.409474
...,...,...
172580,690334,0.112867
172581,690335,0.519712
172582,690336,0.250699
172583,690337,0.128773


Kaggle Machine Learning Competition - Predicting Road Accident Risk

Predicting likelihood of road accidents based on different environmental and road conditions.

Score : 0.05569 (RMSE)

https://www.kaggle.com/competitions/playground-series-s5e10/overview

In [27]:
import joblib
joblib.dump(pipeline, 'road_accident_risk_model.pkl')
print("Model saved as 'road_accident_risk_model.pkl'")

Model saved as 'road_accident_risk_model.pkl'
